In [ ]:
#pip install typing_extensions==4.7.1 --upgrade

In [ ]:
#pip install spacy

In [ ]:
#!python -m spacy download pt
!python -m spacy download pt_core_news_lg

In [ ]:
pip install import-ipynb


In [ ]:
pip install rank-bm25

In [ ]:
import re
import pandas as pd
import numpy as np
import import_ipynb
import util as u
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
pd.options.mode.chained_assignment = None
import warnings
import random
import string
from rank_bm25 import BM25Okapi
import matplotlib as plt
import spacy
nlp = spacy.load('pt_core_news_lg')

warnings.filterwarnings("ignore")

In [ ]:
def get_index_materia(DataFrameMaterias,CodigoMateria):
    return DataFrameMaterias[DataFrameMaterias.CodigoMateria == int(CodigoMateria)].index.values[0]

In [ ]:
def is_materia_tramitando(CodigoMateria):
    url = "https://legis.senado.leg.br/dadosabertos/materia/situacaoatual/%d" % CodigoMateria
    df = u.parse_XML(url,["Tramitando"],"Materias/Materia")
    tramitando = df["Tramitando"].item()
    if tramitando == "Sim":
        return 1
    return 0

In [ ]:
def get_materia_from_index(DataFrameMaterias,index):    
    return DataFrameMaterias.iloc[[index]][['CodigoMateria','Identificacao']]

In [ ]:
def get_lista_similares(DataFrameCosine,DataFrameMaterias,CodigoMateria):
    
    indice = get_index_materia(DataFrameMaterias,CodigoMateria)
    similares = list(enumerate(DataFrameCosine[indice]))
    similares = sorted(similares, key=lambda x:x[1], reverse=True)
    materias = pd.DataFrame()
    achou = 0    
    for similar in similares:        
        materia = get_materia_from_index(DataFrameMaterias,similar[0])               
        materia['similaridade'] = similar[1]
        materias = pd.concat([materias,materia])
        achou = achou+1
        if achou>5:
            break
    #materias = materias[materias['CodigoMateria'] != CodigoMateria]
    return materias

In [ ]:
def get_lista_similares_tramitando(DataFrameCosine,DataFrameMaterias,CodigoMateria):
    
    indice = get_index_materia(DataFrameMaterias,CodigoMateria)
    similares = list(enumerate(DataFrameCosine[indice]))
    similares = sorted(similares, key=lambda x:x[1], reverse=True)
    materias = pd.DataFrame()
    achou = 0    
    for similar in similares:        
        materia = get_materia_from_index(DataFrameMaterias,similar[0])               
        materia['similaridade'] = similar[1]
        ident = materia['Identificacao'].values[0]
        if ((is_materia_tramitando(materia['CodigoMateria']) == 1)  & ident.startswith("P")):
            materias = pd.concat([materias,materia])
            achou = achou+1
            if achou>5:
                break
    #materias = materias[materias['CodigoMateria'] != CodigoMateria]
    return materias

In [ ]:
def get_lista_similares_tramitando_base(DataFrameCosine,DataFrameMaterias,CodigoMateria,DataFrameMateriasAlvo):
    indice = get_index_materia(DataFrameMaterias,CodigoMateria)
    similares = list(enumerate(DataFrameCosine[indice]))
    similares = sorted(similares, key=lambda x:x[1], reverse=True)
    materias = pd.DataFrame()
    achou = 0    
    for similar in similares:        
        materia = get_materia_from_index(DataFrameMaterias,similar[0])               
        materia['similaridade'] = similar[1]
        ident = materia['Identificacao'].values[0]        

        if ((is_materia_tramitando(materia['CodigoMateria']) == 1)  & ident.startswith("P")) & materia.CodigoMateria.isin(DataFrameMateriasAlvo.CodigoMateria).bool():
            materias = pd.concat([materias,materia])
            achou = achou+1
            if achou>5:
                break
        #materias = materias[materias['CodigoMateria'] != CodigoMateria]
    return materias

In [ ]:
def get_lista_similares_tramitando_base_2(DataFrameCosine,DataFrameMaterias,CodigoMateria,DataFrameMateriasAlvo):
    
    indice = get_index_materia(DataFrameMaterias,CodigoMateria)    
    similares = list(enumerate(DataFrameCosine[indice]))    
    similares = sorted(similares, key=lambda x:x[1], reverse=True)        
    df_similares = pd.DataFrame(similares, columns=['Indice', 'ValorSimilaridade'])    
    df_similares = df_similares.set_index('Indice')    
    DataFrameMaterias_filtrado = DataFrameMaterias.loc[df_similares.index]    
    DataFrameMaterias_filtrado['ValorSimilaridade'] = df_similares['ValorSimilaridade']
    # DataFrameMaterias_filtrado agora contém as linhas filtradas com a coluna de similaridade

    #Filtra para conter apenas os valores contidos em DataFrameMateriasAlvo
    condicao = DataFrameMaterias_filtrado['CodigoMateria'].isin(DataFrameMateriasAlvo['CodigoMateria'])
    DataFrameMaterias_filtrado = DataFrameMaterias_filtrado[condicao]

    #Filtra para conter apenas as matérias que começam com P
    DataFrameMaterias_filtrado = DataFrameMaterias_filtrado[DataFrameMaterias_filtrado['Identificacao'].str.startswith('P')]

    
    materias = pd.DataFrame()
    achou = 0;    
    for indice, row in DataFrameMaterias_filtrado.iterrows():
        codigo_materia_pesquisada = row["CodigoMateria"]            
        #esta_tramitando = is_materia_tramitando(codigo_materia_pesquisada)
        esta_tramitando = row["Tramitando"]
        #print('Está:'+esta_tramitando)
        if (esta_tramitando == "Sim"):             
            #materias = materias.append(row)            
            #print(pd.DataFrame(row).T)
            materias = pd.concat([materias,pd.DataFrame(row).T])
            achou = achou + 1    

        if achou >= 5:            
            break  
    
    materias = materias[materias['CodigoMateria'] != CodigoMateria]
    return materias

In [ ]:
def get_materias_similares_bm25(DataFrameMateriasOriginal,QueryString, Bm25Inicializado, DataFrameAlvo):    
    bm25 = Bm25Inicializado
    query = QueryString
    
    query = query.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower()
    tokenized_query = query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    descending_indices = (-bm25_scores).argsort()
    dataframe_filtrado = DataFrameMateriasOriginal.filter(items = descending_indices, axis=0)
    dataframe_filtrado = dataframe_filtrado[dataframe_filtrado['Identificacao'].str.startswith('P')]
    achou = 0
    total_escapa = 0
    df_retorno = pd.DataFrame()
    for index, row in dataframe_filtrado.iterrows():
        
        if DataFrameAlvo['CodigoMateria'].isin([row['CodigoMateria']]).any():        
            achou = achou + 1
            #print('Achei,',row['CodigoMateria'])
            df_retorno = pd.concat([df_retorno,pd.DataFrame(row).T])
        total_escapa = total_escapa + 1
        
        if achou >= 5:
            break
        if total_escapa > 1000:
            print('escapei achando',achou)
            break
    return df_retorno    

In [ ]:
def get_situacao_materia(codigo_materia):
    url = "https://legis.senado.leg.br/dadosabertos/materia/situacaoatual/%d" % codigo_materia
    df = u.parse_XML(url,["Autuacoes/Autuacao/Situacoes/Situacao/DescricaoSituacao","Autuacoes/Autuacao/Local/SiglaLocal"],"Materias/Materia/SituacaoAtual")
    df.rename(columns = {'Autuacoes/Autuacao/Situacoes/Situacao/DescricaoSituacao' : 'Situacao'}, inplace = True)
    df.rename(columns = {'Autuacoes/Autuacao/Local/SiglaLocal' : 'Local'}, inplace = True)
    df['CodigoMateria'] = codigo_materia
    df.set_index('CodigoMateria')
    return df

In [ ]:
#define a function for preprocessing
def limpar_texto(text):    
    text = text.lower() #changes to lower case
    doc = nlp(text)
    #cleaned_tokens = [token.lemma_ for token in doc if token.pos_ == "NOUN" and token.text not in lista_stop_words and token.text not in string.punctuation and not token.text.isnumeric()]
    cleaned_tokens = [token.lemma_ for token in doc if token.is_alpha and (token.pos_ == "NOUN" or token.pos_ == "VERB")]    
    cleaned_text = " ".join(cleaned_tokens)
    return cleaned_text

In [ ]:
get_situacao_materia(145390)

### Leitura  das matérias já alimentadas com os detalhes

In [ ]:
df_detalhes_materias = pd.read_csv("df_detalhes_materias.csv",sep=",")
#df_detalhes_materias = df_detalhes_materias.set_index('CodigoMateria')

In [ ]:
df_detalhes_materias

### Junção das características em uma única coluna

In [ ]:
#Cria a coluna combinacao com a junção das propriedades
df_detalhes_materias['combinacao'] = ''
propriedades = ['Ementa', 'Indexacao', 'Natureza', 'Classe','ClasseHierarquica']
for p in propriedades:
    df_detalhes_materias[p] = df_detalhes_materias[p].fillna('')
    df_detalhes_materias['combinacao'] = df_detalhes_materias['combinacao'] + df_detalhes_materias[p]


In [ ]:
lista_stop_words = stopwords.words('portuguese') + ['lei','altera','modifica']

In [ ]:
lista_stop_words

In [ ]:
# Removendo acentos,caracteres especiais e números
df_detalhes_materias['combinacao'] = df_detalhes_materias['combinacao'].str.replace('[,.:;!?]+', ' ', regex=True).copy()
df_detalhes_materias['combinacao'] = df_detalhes_materias['combinacao'].str.replace ('[/<>()|\+\-\$%&#@\'\"]+', ' ', regex=True).copy()
df_detalhes_materias['combinacao'] = df_detalhes_materias['combinacao'].str.replace('[0-9]+', '', regex=True).copy()
df_detalhes_materias['combinacao'] = df_detalhes_materias['combinacao'].str.lower()

In [ ]:
df_detalhes_materias['combinacao_clean'] = df_detalhes_materias['combinacao'].apply(limpar_texto)

In [ ]:
df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == 236]

In [ ]:
df_detalhes_materias.to_csv('df_detalhes_materias.csv',index = False)

### Cálculo da similaridade com texto sem lematização

In [ ]:
# Contagem da frequência das palavras, eliminando números e caracteres especiais
#cv = CountVectorizer(token_pattern="[^\W\d_]+")
#cv = CountVectorizer(stop_words=lista_stop_words)
cv = TfidfVectorizer(stop_words=lista_stop_words, use_idf=True)
count_matrix = cv.fit_transform(df_detalhes_materias["combinacao"])
cosine_sim_normal = cosine_similarity(count_matrix)
cosine_sim_normal = cosine_sim_normal.astype(np.float32)

### Cálculo da similaridade com texto lemmatizado

In [ ]:
# Contagem da frequência das palavras, eliminando números e caracteres especiais
#cv = CountVectorizer(token_pattern="[^\W\d_]+")
#cv = CountVectorizer(stop_words=lista_stop_words)
cv = TfidfVectorizer(stop_words=lista_stop_words, use_idf=True)
count_matrix = cv.fit_transform(df_detalhes_materias["combinacao_clean"])
cosine_sim = cosine_similarity(count_matrix)
cosine_sim = cosine_sim.astype(np.float32)

In [ ]:
cosine_sim

In [ ]:
# Verificar o vetor contador de palavras
pd.DataFrame(count_matrix.toarray(),columns=cv.get_feature_names_out ())

In [ ]:
#df_similaridade = pd.DataFrame(cosine_sim, 
#             index=df_detalhes_materias.index, columns=df_detalhes_materias.index)
df_similaridade = pd.DataFrame(cosine_sim)

In [ ]:
df_similaridade

In [ ]:
df_similaridade.to_csv('df_similaridade.csv')

## Cálculo de similaridade usando bi-grama com texto sem lematização

In [ ]:
cv = CountVectorizer(stop_words=lista_stop_words)
count_matrix = cv.fit_transform(df_detalhes_materias["combinacao"])
cosine_sim_bigrama_normal = cosine_similarity(count_matrix)
cosine_sim_bigrama_normal = cosine_sim_bigrama_normal.astype('float32')

## Cálculo de similaridade usando bi-grama com texto lemmatizado

In [ ]:
df_detalhes_materias

In [ ]:
cv = CountVectorizer(stop_words=lista_stop_words)
count_matrix = cv.fit_transform(df_detalhes_materias["combinacao_clean"])
cosine_sim_bigrama = cosine_similarity(count_matrix)
cosine_sim_bigrama = cosine_sim_bigrama.astype('float32')

In [ ]:
cosine_sim_bigrama

### Teste da similaridade

In [ ]:
# teste com similaridade de cosseno, unigrama e lematização
codigo_materia = 139268
get_lista_similares(cosine_sim,df_detalhes_materias,codigo_materia)       

In [ ]:
# teste com similaridade de cosseno, bigrama e lematização
codigo_materia = 139268
get_lista_similares(cosine_sim_bigrama,df_detalhes_materias,codigo_materia)       

In [ ]:
# teste com similaridade de cosseno, unigrama sem lematização
codigo_materia = 139268
get_lista_similares(cosine_sim_normal,df_detalhes_materias,codigo_materia)       

In [ ]:
# teste com similaridade de cosseno, bigrama sem lematização
codigo_materia = 139268
get_lista_similares(cosine_sim_bigrama_normal,df_detalhes_materias,codigo_materia)       

### Recomendações ao parlamentar

In [ ]:
df_interesse = pd.read_csv("interesse.csv",sep=",")

In [ ]:
df_interesse

In [ ]:
df_aux_interesse = df_interesse.set_index("CodigoMateria")
df_aux_detalhes = df_detalhes_materias.set_index("CodigoMateria")
df_aux_join = df_aux_detalhes.join(df_aux_interesse)
df_aux_join

In [ ]:
# RODRIGO CUNHA
#codigo_parlamentar = 5905

# HEINZE
#codigo_parlamentar = 1186

#GIRÃO
#codigo_parlamentar = 5976

#PLINIO
codigo_parlamentar = 5502

df_dez_interessantes = df_aux_join.query('CodigoParlamentar == %d & Identificacao.str.startswith("P") & not ClasseHierarquica.str.contains("Honorífico") &  not ClasseHierarquica.str.contains("Orçamento Anual") &  not Natureza.str.contains("Concessão")' % codigo_parlamentar).sort_values("interesse",ascending=False).head(10)
#df_interesse = df_interesse.set_index('CodigoMateria')
df_top_similares = pd.DataFrame()
for index,row in df_dez_interessantes.iterrows():
    codigo_materia = index
    df_materia_pesquisada = df_detalhes_materias.query('CodigoMateria == %d' % codigo_materia)  
    
    #print (codigo_materia)
    #print(df_materia_pesquisada)
    print ('\n*********************\nSimilares à matéria %s - codigo %d  - %s\n********************\n' % (df_materia_pesquisada['Identificacao'].values[0],codigo_materia,df_materia_pesquisada['Ementa'].values[0]) )
    similares_pesquisadas = get_lista_similares_tramitando(cosine_sim,df_detalhes_materias,codigo_materia) 
    similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]
    df_top_similares = pd.concat([df_top_similares,similares_pesquisadas])
    print(similares_pesquisadas.head(5))

print ('\n*********************\nMatérias de mais similaridade para o parlamentar \n********************\n' )    
df_top_similares.sort_values("similaridade",ascending=False).head(10)

In [ ]:
# RODRIGO CUNHA
#codigo_parlamentar = 5905

# HEINZE
#codigo_parlamentar = 1186

#GIRÃO
#codigo_parlamentar = 5976

#PLINIO
codigo_parlamentar = 5502

df_dez_interessantes = df_aux_join.query('CodigoParlamentar == %d & Identificacao.str.startswith("P") & not ClasseHierarquica.str.contains("Honorífico") &  not ClasseHierarquica.str.contains("Orçamento Anual") &  not Natureza.str.contains("Concessão")' % codigo_parlamentar).sort_values("interesse",ascending=False).head(5)
#df_interesse = df_interesse.set_index('CodigoMateria')
df_top_similares = pd.DataFrame()
i=0
print('<html> <head> <meta charset="utf-8" /> <meta http-equiv="X-UA-Compatible" content="IE=edge"> <title>Sugestões</title> <meta name="viewport" content="width=device-width, initial-scale=1"> <script defer src="https://use.fontawesome.com/releases/v5.1.0/js/all.js" integrity="sha384-3LK/3kTpDE/Pkp8gTNp2gR/2gOiwQ6QaO7Td0zV76UFJVhqLl4Vl3KL1We6q6wR9" crossorigin="anonymous"></script> <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css"> <link rel="stylesheet" href="netflix.css"> <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.6.3/jquery.min.js"></script> <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/js/bootstrap.min.js"></script> </head> <body> <div class="wrapper"> <header> <nav class="main-nav"> <h2 style="color: red;">Sugestões para o Senador Rodrigo Cunha</h2> </nav> </header>')
print('') 
print('') 
print('') 
for index,row in df_dez_interessantes.iterrows():
    
    
    codigo_materia = index
    df_materia_pesquisada = df_detalhes_materias.query('CodigoMateria == %d' % codigo_materia)    
    
    
    #detalhes
    df_detalhes_completo = u.get_detalhes_materia_completo(codigo_materia)    
    identificacao = df_detalhes_completo['Identificacao']
    autor = df_detalhes_completo['Autor']
    link = df_detalhes_completo['link']
    ementa = df_detalhes_completo['Ementa']
    explicacao = df_detalhes_completo['ExplicacaoEmenta']
    if(explicacao[0] is None):
       explicacao[0] = ''
    ####
    i = i+1
    print('<h3 id="myList">Porque você atuou no '+identificacao[0]+' <a  data-toggle="modal" href="#m'+str(i)+'" class="btn btn-info btn-sm">'+
'<span class="glyphicon glyphicon-info-sign"></span> Saiba mais'+
'</a> </h3> <h5>Ementa: '+ementa[0]+'</h5>')
    
    print('<div class="modal fade" id="m'+str(i)+'" role="dialog"> <div class="modal-dialog"> <!-- Modal content--> <div class="modal-content"> <div class="modal-header" style="padding:35px 50px;"> <button type="button" class="close" data-dismiss="modal">&times;</button> <h4 class="modal-title">'+identificacao[0]+' </h4> </div> <div class="modal-body"> <b>Autor:</b> '+autor[0]+' <br><br> <b>Ementa:</b> '+ementa[0]+' <br><br> <b>Explicação da ementa:</b>'+explicacao[0]+' <br><br> <a href="'+link[0]+'" class="btn btn-primary btn-sm linkmateria" target="_blank"> Página da matéria</a> </div> <div class="modal-footer"> <button type="button" class="btn btn-default" data-dismiss="modal">Close</button> </div> </div> </div> </div>')
    
    similares_pesquisadas = get_lista_similares_tramitando(cosine_sim,df_detalhes_materias,codigo_materia) 
    similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]
    similares_pesquisadas = similares_pesquisadas.set_index('CodigoMateria')
    df_top_similares = pd.concat([df_top_similares,similares_pesquisadas])
    print('')  
    print('<div class="box">')
    for index_pesq,row_pesq in similares_pesquisadas.iterrows():        
        codigo_materia_pesq = index_pesq
        df_detalhes_completo = u.get_detalhes_materia_completo(codigo_materia_pesq)
        identificacao = df_detalhes_completo['Identificacao']
        autor = df_detalhes_completo['Autor']
        link = df_detalhes_completo['link']
        ementa = df_detalhes_completo['Ementa']
        explicacao = df_detalhes_completo['ExplicacaoEmenta']
        if(explicacao[0] is None):
            explicacao[0] = ''
        i = i+1
        print('<a data-toggle="modal" href="#m'+str(i)+'">'+identificacao[0]+'</a>')
        print('<div class="modal fade" id="m'+str(i)+'" role="dialog"> <div class="modal-dialog"> <!-- Modal content--> <div class="modal-content"> <div class="modal-header" style="padding:35px 50px;"> <button type="button" class="close" data-dismiss="modal">&times;</button> <h4 class="modal-title">'+identificacao[0]+' </h4> </div> <div class="modal-body"> <b>Autor:</b> '+autor[0]+' <br><br> <b>Ementa:</b> '+ementa[0]+' <br><br> <b>Explicação da ementa:</b>'+explicacao[0]+' <br><br> <a href="'+link[0]+'" class="btn btn-primary btn-sm linkmateria" target="_blank"> Página da matéria</a> </div> <div class="modal-footer"> <button type="button" class="btn btn-default" data-dismiss="modal">Close</button> </div> </div> </div> </div>')
    print('</div><br><br>')
    print('')
    print('')
    print('')
        

print('<!-- TOP 10 --> <section class="main-container"><div class="location" id="home"><h1 id="home">Você deve gostar de</h1><div class="box">')

df_top_similares = df_top_similares.sort_values("similaridade",ascending=False).head(10)
for index_pesq,row_pesq in df_top_similares.iterrows():
        codigo_materia_pesq = index_pesq
        df_detalhes_completo = u.get_detalhes_materia_completo(codigo_materia_pesq)
        identificacao = df_detalhes_completo['Identificacao']
        autor = df_detalhes_completo['Autor']
        link = df_detalhes_completo['link']
        ementa = df_detalhes_completo['Ementa']
        explicacao = df_detalhes_completo['ExplicacaoEmenta']
        if(explicacao[0] is None):
            explicacao[0] = ''
        i = i+1
        
        print('<a data-toggle="modal" href="#m'+str(i)+'">'+identificacao[0]+'</a>')
        print('<div class="modal fade" id="m'+str(i)+'" role="dialog"> <div class="modal-dialog"> <!-- Modal content--> <div class="modal-content"> <div class="modal-header" style="padding:35px 50px;"> <button type="button" class="close" data-dismiss="modal">&times;</button> <h4 class="modal-title">'+identificacao[0]+' </h4> </div> <div class="modal-body"> <b>Autor:</b> '+autor[0]+' <br><br> <b>Ementa:</b> '+ementa[0]+' <br><br> <b>Explicação da ementa:</b>'+explicacao[0]+' <br><br> <a href="'+link[0]+'" class="btn btn-primary btn-sm linkmateria" target="_blank"> Página da matéria</a> </div> <div class="modal-footer"> <button type="button" class="btn btn-default" data-dismiss="modal">Close</button> </div> </div> </div> </div>')
        
print('</div></div><br><br><br>')
print('')
print('</div> <!-- FOOTER --> <footer> <p>Versão em desenvolvimento</p> <p>Carlos Rafael</p> </footer> </div> </body> </html>')

In [ ]:
df_similares_da_materia.join(df_detalhes_materias_interesse_aux)

In [ ]:
# TESTAR SIMILARIDADS 
DataFrameMaterias = df_detalhes_materias
DataFrameCosine = cosine_sim
CodigoMateria = 151530   

indice = get_index_materia(DataFrameMaterias,CodigoMateria)
similares = list(enumerate(DataFrameCosine[indice]))
similares = sorted(similares, key=lambda x:x[1], reverse=True)
similares


In [ ]:
#materia = get_materia_from_index(DataFrameMaterias,7927)
#identificacao = materia['Identificacao']
identificacao = 'AVS 39/2018'
codigo_materia = 134595
#is_materia_tramitando(codigo_materia) == 1
if (is_materia_tramitando(codigo_materia) == 1 & np.where(identificacao.startswith("P"),True,False)):
    print(is_materia_tramitando(codigo_materia) == 1)
#if (np.where(identificacao != 'OPA', True, False)):
#    print('opa')
#print (identificacao == 'OPA')
#print(str(identificacao))
#if identificacao == 'OPA':
#    print('opa')
#else:
#    print('nao - %s' % identificacao.value)
#if (is_materia_tramitando(codigo_materia) == 1  & np.where(identificacao.startswith("P"),True,False)):
#   print('opa')
#else:
#    print(identificacao)

In [ ]:
codigo_materia = 152146
materias = get_lista_similares_tramitando(cosine_sim,df_detalhes_materias,codigo_materia) 
materias

In [ ]:
DataFrameMaterias = df_detalhes_materias
CodigoMateria = 152146
DataFrameCosine = cosine_sim
indice = get_index_materia(DataFrameMaterias,CodigoMateria)
similares = list(enumerate(DataFrameCosine[indice]))
similares = sorted(similares, key=lambda x:x[1], reverse=True)
materias = pd.DataFrame()
achou = 0    
for similar in similares:        
    materia = get_materia_from_index(DataFrameMaterias,similar[0])               
    materia['similaridade'] = similar[1]
    ident = materia['Identificacao'].values[0]
    if ((is_materia_tramitando(materia['CodigoMateria']) == 1) & ident.startswith("P")):
        print ("OK - "+ident)
        materias = pd.concat([materias,materia])
        achou = achou+1
        if achou>15:
            break
    else:
        print ("NÃO - "+ident)

In [ ]:
DataFrameMaterias = df_detalhes_materias
CodigoMateria = 139840
DataFrameCosine = cosine_sim
indice = get_index_materia(DataFrameMaterias,CodigoMateria)
DataFrameCosine[indice]
#df_similaridade
teste_df = pd.DataFrame(count_matrix.toarray(),columns=cv.get_feature_names_out ())


In [ ]:
#indice
teste_df.loc[5987,:]

In [ ]:
teste_df.loc[5987,:].to_csv('teste.csv')

In [ ]:
df = u.get_detalhes_materia_completo(156011)
df

In [ ]:
lista_stop_words = stopwords.words('portuguese')
lista_stop_words

In [ ]:
df_similaridade_teste = pd.read_csv("df_similaridade.csv",sep=",")

In [ ]:
df_similaridade_teste.head(5).iloc[:, : 5]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import table

ax = plt.subplot(111, frame_on=False)
ax.xaxis.set_visible(False) # hide the x axis
ax.yaxis.set_visible(False) # hide the y axis
table(ax, df_similaridade_teste.head(5).iloc[:, : 5], loc='center') # where df is your data frame
#plt.savefig('mytable.png')

#ax = plt.subplot(111, frame_on=False) # no visible frame
#ax.xaxis.set_visible(False) # hide the x axis
#ax.yaxis.set_visible(False) # hide the y axis
#table(ax, df, loc='center') # where df is your data frame
#plt.savefig('mytable.png')

In [ ]:
df_similaridade_teste

# Testes de Eficiência/RC

In [ ]:
df_detalhes_materias

In [ ]:
df_interesse

In [ ]:
df_interesse = df_interesse.set_index('CodigoMateria')

In [ ]:
df_interesse_rc = df_interesse[df_interesse['CodigoParlamentar'] == 5905]

In [ ]:
df_interesse_rc = df_interesse_rc.join(df_detalhes_materias.set_index("CodigoMateria"),how='left')[['CodigoParlamentar','interesse','Ementa','Identificacao','Natureza','ClasseHierarquica']]

In [ ]:
df_interesse_rc = df_interesse_rc.dropna()

In [ ]:
df_interesse_rc['ano_materia'] = df_interesse_rc['Identificacao'].str[-4:]
df_interesse_rc['ano_materia'] = pd.to_numeric(df_interesse_rc['ano_materia'], errors='coerce')
df_interesse_rc.dropna(subset=['ano_materia'], inplace=True)
df_interesse_rc = df_interesse_rc.astype({"ano_materia":"int"})

In [ ]:
df_interesse_rc.groupby(['ano_materia'])['ano_materia'].count()

In [ ]:
df_interesse_rc_treino = df_interesse_rc[df_interesse_rc['ano_materia'] <= 2021]

In [ ]:
df_interesse_rc_treino.sort_values(by=['interesse'], ascending=False)

In [ ]:
df_interesse_rc_treino.query('Identificacao.str.startswith("P") & not ClasseHierarquica.str.contains("Honorífico") &  not ClasseHierarquica.str.contains("Orçamento Anual") &  not Natureza.str.contains("Concessão")').sort_values("interesse",ascending=False).head(10)

In [ ]:
df_interesse_rc_verdade = df_interesse_rc[df_interesse_rc['ano_materia'] >= 2022]

In [ ]:
df_interesse_rc_verdade.sort_values(by=['interesse'], ascending=False)

In [ ]:
df_detalhes_materias_aux = df_detalhes_materias
df_detalhes_materias_aux['ano_materia'] = df_detalhes_materias_aux['Identificacao'].str[-4:]
#df_detalhes_materias_aux = df_detalhes_materias_aux.astype({"ano_materia":"int"}, errors='coerce')
df_detalhes_materias_aux['ano_materia'] = pd.to_numeric(df_detalhes_materias_aux['ano_materia'], errors='coerce')
df_detalhes_materias_aux.dropna(subset=['ano_materia'], inplace=True)
df_detalhes_materias_aux = df_detalhes_materias_aux.astype({"ano_materia":"int"}, errors='raise')
df_detalhes_materias_aux
df_detalhes_materias_aux = df_detalhes_materias_aux[df_detalhes_materias_aux['ano_materia'] >= 2022]

In [ ]:
df_detalhes_materias_aux

In [ ]:
# Contagem da frequência das palavras, eliminando números e caracteres especiais
#cv = CountVectorizer(token_pattern="[^\W\d_]+")
cv_aux = CountVectorizer(stop_words=lista_stop_words)
count_matrix_aux = cv_aux.fit_transform(df_detalhes_materias_aux["combinacao"])
cosine_sim = cosine_similarity(count_matrix)
pd.DataFrame(count_matrix.toarray(),columns=cv.get_feature_names_out ())
df_similaridade = pd.DataFrame(cosine_sim)
df_similaridade

## Criando recomendações para as matérias de 2022

In [ ]:
df_interesse_rc_treino

In [ ]:
df_detalhes_materias_teste

In [ ]:
df_aux_interesse

In [ ]:
df_aux_detalhes

In [ ]:
# cria a base de testes, com matérias a partir de 2022
df_detalhes_materias_teste = pd.DataFrame(df_detalhes_materias)
df_detalhes_materias_teste['ano_materia'] = df_detalhes_materias_teste['Identificacao'].str[-4:]
df_detalhes_materias_teste['ano_materia'] = pd.to_numeric(df_detalhes_materias_teste['ano_materia'], errors='coerce')
df_detalhes_materias_teste.dropna(subset=['ano_materia'], inplace=True)
df_detalhes_materias_teste = df_detalhes_materias_teste.astype({"ano_materia":"int"}, errors='raise')
df_detalhes_materias_teste = df_detalhes_materias_teste[df_detalhes_materias_teste['ano_materia'] >= 2022] 

In [ ]:
codigo_parlamentar = 5905


#Pega as 10 mais interessantes dos dados de TREINO (Até 2021)
df_dez_interessantes = df_interesse_rc_treino.query('CodigoParlamentar == %d & Identificacao.str.startswith("P") & not ClasseHierarquica.str.contains("Honorífico") &  not ClasseHierarquica.str.contains("Orçamento Anual") &  not Natureza.str.contains("Concessão")' % codigo_parlamentar).sort_values("interesse",ascending=False).head(10)

# Procura os similares na base de teste (A partir de 2022)
df_top_similares_teste = pd.DataFrame()
for index,row in df_dez_interessantes.iterrows():
    codigo_materia = index
    df_materia_pesquisada = df_detalhes_materias.query('CodigoMateria == %d' % codigo_materia)    
    
    
    #print(codigo_materia)
    similares_pesquisadas = get_lista_similares_tramitando_base(cosine_sim,df_detalhes_materias,codigo_materia, df_detalhes_materias_teste) 
    similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]
    similares_pesquisadas = similares_pesquisadas.set_index('CodigoMateria')    
    df_top_similares_teste = pd.concat([df_top_similares_teste,similares_pesquisadas])
#df_top_similares_teste = df_top_similares_teste.sort_values("similaridade",ascending=False).head(20)

In [ ]:
df_top_similares_teste

In [ ]:
# Verifica se as matérias sugeridas (df_top_similares_teste) estão na base VERDADE

total_match = 0
for index,row in df_top_similares_teste.iterrows():    
    codigo_materia = index
    #print ("Vou verificar a materia: ",codigo_materia)
    if codigo_materia in df_interesse_rc_verdade.index:
        total_match = total_match + 1
        #df_materia_encontrada = pd.DataFrame(df_interesse_rc_verdade.loc[index].to_frame().T)
        identificacao = df_interesse_rc_verdade.loc[index]['Identificacao']
        interesse = df_interesse_rc_verdade.loc[index]['interesse']
        print("Achei a materia ",codigo_materia,": ",identificacao," com interesse ",interesse)

percentual = round((total_match/len(df_top_similares_teste)*100),2)
print("Acertei ", total_match, "de ", len(df_top_similares_teste), ". Percentual de ",percentual,"%")

## Teste de Eficiência GERAL

In [ ]:
# RODRIGO CUNHA
#codigo_parlamentar_teste = 5905

# HEINZE
#codigo_parlamentar_teste = 1186

#GIRÃO
#codigo_parlamentar_teste = 5976

#PLINIO
#codigo_parlamentar_teste = 5502

#Alan Rick
codigo_parlamentar_teste = 5672

In [ ]:
df_interesse = pd.read_csv("interesse.csv",sep=",")
df_interesse = df_interesse.set_index('CodigoMateria')

In [ ]:
df_interesse_parla = df_interesse[df_interesse['CodigoParlamentar'] == codigo_parlamentar_teste]
df_interesse_parla = df_interesse_parla.join(df_detalhes_materias.set_index("CodigoMateria"),how='left')[['CodigoParlamentar','interesse','Ementa','Identificacao','Natureza','ClasseHierarquica']]
df_interesse_parla = df_interesse_parla.dropna()
df_interesse_parla['ano_materia'] = df_interesse_parla['Identificacao'].str[-4:]
df_interesse_parla['ano_materia'] = pd.to_numeric(df_interesse_parla['ano_materia'], errors='coerce')
df_interesse_parla.dropna(subset=['ano_materia'], inplace=True)
df_interesse_parla = df_interesse_parla.astype({"ano_materia":"int"})

In [ ]:
df_interesse_parla_treino = df_interesse_parla[df_interesse_parla['ano_materia'] <= 2021]
df_interesse_parla_verdade = df_interesse_parla[df_interesse_parla['ano_materia'] >= 2022]

In [ ]:
df_interesse_parla

In [ ]:
df_interesse_parla_treino

In [ ]:
df_interesse_parla_verdade

### Criando recomendações no TESTE DE EFICIÊNCIA GERAL

In [ ]:
# cria a base de testes, com matérias a partir de 2022
df_detalhes_materias_teste = pd.DataFrame(df_detalhes_materias)
df_detalhes_materias_teste['ano_materia'] = df_detalhes_materias_teste['Identificacao'].str[-4:]
df_detalhes_materias_teste['ano_materia'] = pd.to_numeric(df_detalhes_materias_teste['ano_materia'], errors='coerce')
df_detalhes_materias_teste.dropna(subset=['ano_materia'], inplace=True)
df_detalhes_materias_teste = df_detalhes_materias_teste.astype({"ano_materia":"int"}, errors='raise')
df_detalhes_materias_teste = df_detalhes_materias_teste[df_detalhes_materias_teste['ano_materia'] >= 2022] 


In [ ]:
df_detalhes_materias_teste

In [ ]:
df_autorias = pd.read_csv("df_autorias.csv",sep=",")
df_autorias_parla = df_autorias[df_autorias['CodigoParlamentar'] == codigo_parlamentar_teste]
#df_autorias_parla

In [ ]:
# Suponha que você já tenha os DataFrames df_detalhes_materias_teste e df_autorias_parla

# Crie uma lista dos valores únicos da coluna "CodigoMateria" em df_autorias_parla
codigos_autorias_parla = df_autorias_parla['CodigoMateria'].unique()

# Filtrar os registros em df_detalhes_materias_teste que não estão em df_autorias_parla
df_detalhes_materias_teste = df_detalhes_materias_teste[~df_detalhes_materias_teste['CodigoMateria'].isin(codigos_autorias_parla)]


In [ ]:
df_detalhes_materias_teste

In [ ]:
# Seleciona as 10 mais interessantes dos dados de TREINO (Até 2021)
condicao = (
    (df_interesse_parla_treino['CodigoParlamentar'] == codigo_parlamentar_teste) &
    (df_interesse_parla_treino['Identificacao'].str.startswith('P')) &
    (~df_interesse_parla_treino['ClasseHierarquica'].str.contains('Honorífico|Orçamento Anual')) &
    (~df_interesse_parla_treino['Natureza'].str.contains('Concessão'))
)

df_interessantes = df_interesse_parla_treino[condicao].sort_values('interesse', ascending=False).head(30)

# Procura os similares na base de teste (A partir de 2022)
df_top_similares_teste = pd.DataFrame()
for codigo_materia in df_interessantes.index:
    df_materia_pesquisada = df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == codigo_materia]
    similares_pesquisadas = get_lista_similares_tramitando_base_2(cosine_sim, df_detalhes_materias, codigo_materia, df_detalhes_materias_teste)
    similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]
    similares_pesquisadas = similares_pesquisadas.set_index('CodigoMateria')
    df_top_similares_teste = pd.concat([df_top_similares_teste, similares_pesquisadas])

df_top_similares_teste = df_top_similares_teste[~df_top_similares_teste.index.duplicated(keep='first')]

In [ ]:
df_interessantes

### Verificando a eficiência

In [ ]:
df_interesse_parla_verdade

In [ ]:
df_top_similares_teste

In [ ]:
# Verifica se as matérias sugeridas (df_top_similares_teste) estão na base VERDADE

total_match = 0
for index,row in df_top_similares_teste.iterrows():    
    codigo_materia = index
    #print ("Vou verificar a materia: ",codigo_materia)
    if codigo_materia in df_interesse_parla_verdade.index:
        total_match = total_match + 1        
        identificacao = df_interesse_parla_verdade.loc[index]['Identificacao']
        interesse = df_interesse_parla_verdade.loc[index]['interesse']        
        print("Achei a materia ",codigo_materia,": ",identificacao," com interesse ",interesse, )

percentual = round((total_match/len(df_top_similares_teste)*100),2)
print("Acertei ", total_match, "de ", len(df_top_similares_teste), ". Percentual de ",percentual,"%")

# Teste de eficiência para todos os senadores

In [ ]:
#Só na primeira vez
df_interesse = pd.read_csv("interesse.csv",sep=",")
df_interesse = df_interesse.set_index('CodigoMateria')

df_senadores = pd.read_csv("df_senadores.csv",sep=",")
df_senadores = df_senadores.set_index('CodigoParlamentar')
df_senadores['PegouDetalhe'] = False

df_stat = pd.DataFrame(columns=[
    'CodigoParlamentar',
    'NomeParlamentar',
    'TotalBaseTreino',
    'TotalBaseVerdade',    
    'AcertosUnigramaLema',
    'PercentualUnigramaLema',
    'PercentualCoberturaUnigramaLema',
    'AcertosBigramaLema',
    'PercentualBigramaLema',
    'PercentualCoberturaBigramaLema',
    'AcertosUnigramaNormal',
    'PercentualUnigramaNormal',
    'PercentualCoberturaUnigramaNormal',
    'AcertosBigramaNormal',
    'PercentualBigramaNormal',
    'PercentualCoberturaBigramaNormal',
    'AcertosBM25Lema',
    'PercentualBM25Lema',
    'PercentualBM25CoberturaLema'
    'AcertosBM25Normal',
    'PercentualBM25Normal',
    'PercentualCoberturaBM25Normal',
    'AcertosAleatorios',
    'PercentualAleatorios',
    'PercentualAleatoriosCobertura'
])

In [ ]:
df_interessantes[df_interessantes.index == 159777]

In [ ]:
df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == 159777]

In [ ]:
df_senadores[(df_senadores['PegouDetalhe'] == False) & (df_senadores.index == 5672)]

In [ ]:
#Transformando as colunas de df_stat para numericas
columns = ['TotalBaseTreino',
    'TotalBaseVerdade',    
    'AcertosUnigramaLema',
    'PercentualUnigramaLema',
    'PercentualCoberturaUnigramaLema',
    'AcertosBigramaLema',
    'PercentualBigramaLema',
    'PercentualCoberturaBigramaLema',
    'AcertosUnigramaNormal',
    'PercentualUnigramaNormal',
    'PercentualCoberturaUnigramaNormal',
    'AcertosBigramaNormal',
    'PercentualBigramaNormal',
    'PercentualCoberturaBigramaNormal',
    'AcertosBM25Lema',
    'PercentualBM25Lema',
    'PercentualBM25CoberturaLema'
    'AcertosBM25Normal',
    'PercentualBM25Normal',
    'PercentualCoberturaBM25Normal',
    'AcertosAleatorios',
    'PercentualAleatorios',
    'PercentualAleatoriosCobertura'] 
df_stat[columns] = df_stat[columns].apply(lambda x: x.astype('float'))

In [ ]:
#CUIDADO PRA NÃO SOBRESCREVER
df_stat.to_csv('df_stat.csv')

## Estatísticas do resultado

In [ ]:
df_stat

In [ ]:
df_stat.info()

In [ ]:
#Usando código velho
df_stat.describe()

In [ ]:
#Usando código otimizado
df_stat.describe()

In [ ]:
len(df_stat[df_stat['TotalBaseTreino'] > 100])

In [ ]:
df_stat[df_stat['TotalBaseTreino'] > 100].describe()

In [ ]:
df_stat[((df_stat['TotalBaseVerdade'] > 10) and (df_stat['TotalBaseVerdade'] < 30))].describe()

In [ ]:
len(df_stat[df_stat['TotalSugestoes'] > 10])

In [ ]:
df_stat[df_stat['TotalSugestoes'] > 10].describe()

## Sugestoes dos senadores

In [ ]:
#Só na primeira vez
df_interesse = pd.read_csv("interesse.csv",sep=",")
df_interesse = df_interesse.set_index('CodigoMateria')

df_senadores = pd.read_csv("df_senadores.csv",sep=",")
df_senadores = df_senadores.set_index('CodigoParlamentar')
df_senadores['PegouDetalhe'] = False

df_stat = pd.DataFrame(columns=[
    'CodigoParlamentar',
    'NomeParlamentar',
    'TotalBaseTreino',
    'TotalBaseVerdade',
    'TotalSugestoes',
    'AcertosUnigramaLema',
    'PercentualUnigramaLema',
    'PercentualCoberturaUnigramaLema',
    'AcertosBigramaLema',
    'PercentualBigramaLema',
    'PercentualCoberturaBigramaLema',
    'AcertosUnigramaNormal',
    'PercentualUnigramaNormal',
    'PercentualCoberturaUnigramaNormal',
    'AcertosBigramaNormal',
    'PercentualBigramaNormal',
    'PercentualCoberturaBigramaNormal',
    'AcertosBM25Lema',
    'PercentualBM25BM25Lema',
    'PercentualBM25CoberturaBM25Lema'
    'AcertosBM25Normal',
    'PercentualBM25Normal',
    'PercentualCoberturaBM25Normal'    
])

In [ ]:
df_detalhes_materias["combinacao"]

In [ ]:
#CÓDIGO PARA SUPER SUGESTOES
df_senadores_aux = df_senadores[(df_senadores['PegouDetalhe'] == False) & (df_senadores.index == 5905)]
#df_senadores_aux = df_senadores[(df_senadores['PegouDetalhe'] == False)]

#Calcula o bm25 considerando a base de teste, para poder ser utilizado posteriormente para filtrar as mais similares

corpus_parla_normal = df_detalhes_materias["combinacao"]
corpus_parla_normal = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower() for doc in corpus_parla_normal]
tokenized_corpus_parla_normal = [doc.split(" ") for doc in corpus_parla_normal]
bm25_parla_normal = BM25Okapi(tokenized_corpus_parla_normal)  

corpus_parla_lema = df_detalhes_materias["combinacao_clean"]
corpus_parla_lema = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower() for doc in corpus_parla_lema]
tokenized_corpus_parla_lema = [doc.split(" ") for doc in corpus_parla_lema]
bm25_parla_lema = BM25Okapi(tokenized_corpus_parla_lema)  

df_sugestoes_cosseno_geral_lema = pd.DataFrame()
df_sugestoes_bigrama_geral_lema = pd.DataFrame()
df_sugestoes_cosseno_geral_normal = pd.DataFrame()
df_sugestoes_bigrama_geral_normal = pd.DataFrame()
df_sugestoes_bm25_geral_lema = pd.DataFrame()
df_sugestoes_bm25_geral_normal = pd.DataFrame()
df_sugestoes_aleatorias_geral = pd.DataFrame()

for index,row in df_senadores_aux.iterrows():
    codigo_parlamentar_teste = index
    #print('codigo:',codigo_parlamentar_teste)
    nome_parlamentar = row['NomeParlamentar']
    
    df_interesse_parla = df_interesse[df_interesse['CodigoParlamentar'] == codigo_parlamentar_teste]
    df_interesse_parla = df_interesse_parla.join(df_detalhes_materias.set_index("CodigoMateria"),how='left')[['CodigoParlamentar','Ementa','Identificacao','Natureza','ClasseHierarquica','interesse','justificativa_interesse']]
    df_interesse_parla = df_interesse_parla.dropna()
        
    condicao = (
        (df_interesse_parla['interesse'] > 0) &
        (df_interesse_parla['Identificacao'].str.startswith('P')) &
        (~df_interesse_parla['ClasseHierarquica'].str.contains('Orçamento Anual')) &
        (~df_interesse_parla['Natureza'].str.contains('Concessão'))
    )
    df_interesse_parla = df_interesse_parla[condicao]
    
   
    #Define a base que será usada para teste    
    df_detalhes_materias_teste = pd.DataFrame(df_detalhes_materias)

    #Remove as autorias da base de teste    
    df_detalhes_materias_teste = df_detalhes_materias_teste[~df_detalhes_materias_teste['CodigoMateria'].isin(codigos_autorias_parla)]
    df_detalhes_materias_teste = df_detalhes_materias_teste[df_detalhes_materias_teste['Tramitando'] == 'Sim']
    
    #Aplica as mesmas condições na base de teste
    condicao = (
        (df_detalhes_materias_teste['Identificacao'].str.startswith('P')) &
        (~df_detalhes_materias_teste['ClasseHierarquica'].str.contains('Orçamento Anual')) &
        (~df_detalhes_materias_teste['Natureza'].str.contains('Concessão'))
    )
    df_detalhes_materias_teste = df_detalhes_materias_teste[condicao]
  
    
    
    # Seleciona as 10 mais interessantes 

    df_interessantes = df_interesse_parla.sort_values('interesse', ascending=False).head(10)

    # 1 - Procura os similares na base de teste, usando similaridade de cosseno
    df_top_similares_cosseno = pd.DataFrame()
    for codigo_materia in df_interessantes.index:        
        df_materia_pesquisada = df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == codigo_materia]        
        similares_pesquisadas = get_lista_similares_tramitando_base_2(cosine_sim, df_detalhes_materias, codigo_materia, df_detalhes_materias_teste)        
        similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]        
        similares_pesquisadas = similares_pesquisadas.set_index('CodigoMateria')        
        df_top_similares_cosseno = pd.concat([df_top_similares_cosseno, similares_pesquisadas])        
    #Elimina as duplicatas
    df_top_similares_cosseno = df_top_similares_cosseno[~df_top_similares_cosseno.index.duplicated(keep='first')]    
    
            
    # 2 - Procura os similares na base de teste usando bigrama e similaridade de cosseno
    df_top_similares_bigrama = pd.DataFrame()
    for codigo_materia in df_interessantes.index:
        df_materia_pesquisada = df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == codigo_materia]
        similares_pesquisadas = get_lista_similares_tramitando_base_2(cosine_sim_bigrama, df_detalhes_materias, codigo_materia, df_detalhes_materias_teste)
        similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]
        similares_pesquisadas = similares_pesquisadas.set_index('CodigoMateria')
        df_top_similares_bigrama = pd.concat([df_top_similares_bigrama, similares_pesquisadas])
    #Elimina as duplicatas
    df_top_similares_bigrama = df_top_similares_bigrama[~df_top_similares_bigrama.index.duplicated(keep='first')]    
    
        
    # 3 - Procura os similares na base de teste, usando BM25
    df_top_similares_bm25 = pd.DataFrame()
    for codigo_materia in df_interessantes.index:
        df_materia_pesquisada = df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == codigo_materia]
        query_parla = df_materia_pesquisada['combinacao'].to_string(index=False)
        similares_pesquisadas = get_materias_similares_bm25(df_detalhes_materias,query_parla,bm25_parla,df_detalhes_materias_teste)
        similares_pesquisadas = similares_pesquisadas[similares_pesquisadas['CodigoMateria'] != codigo_materia]
        similares_pesquisadas = similares_pesquisadas.set_index('CodigoMateria')
        df_top_similares_bm25 = pd.concat([df_top_similares_bm25, similares_pesquisadas])
    #Elimina as duplicatas
    df_top_similares_bm25 = df_top_similares_bm25[~df_top_similares_bm25.index.duplicated(keep='first')]        
       
    #Grava nas sugestões gerais
    df_top_similares_cosseno['CodigoParlamentar'] = codigo_parlamentar_teste
    df_top_similares_bigrama['CodigoParlamentar'] = codigo_parlamentar_teste
    df_top_similares_bm25['CodigoParlamentar'] = codigo_parlamentar_teste
    
    df_sugestoes_cosseno_geral = pd.concat([df_sugestoes_cosseno_geral, df_top_similares_cosseno])  
    df_sugestoes_bigrama_geral = pd.concat([df_sugestoes_bigrama_geral, df_top_similares_bigrama])  
    df_sugestoes_bm25_geral = pd.concat([df_sugestoes_bm25_geral, df_top_similares_bm25])  
    
    df_senadores.at[index, 'PegouDetalhe'] = True


In [ ]:
df_sugestoes_cosseno_geral[df_sugestoes_cosseno_geral['CodigoParlamentar'] == 5905]

In [ ]:
df_detalhes_materias

In [ ]:
df_sugestoes_bm25_geral[df_sugestoes_bm25_geral['CodigoParlamentar'] == 5905]

# Testando o BM25

In [ ]:
def get_materias_similares_bm25(DataFrameMateriasOriginal,QueryString, Bm25Inicializado, DataFrameAlvo):    
    bm25 = Bm25Inicializado
    query = QueryString
    
    query = query.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower()
    tokenized_query = query.split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    descending_indices = (-bm25_scores).argsort()
    dataframe_filtrado = DataFrameMateriasOriginal.filter(items = descending_indices, axis=0)
    achou = 0
    total_escapa = 0
    df_retorno = pd.DataFrame()
    for index, row in dataframe_filtrado.iterrows():
        
        if DataFrameAlvo['CodigoMateria'].isin([row['CodigoMateria']]).any():        
            achou = achou + 1
            #print('Achei,',row['CodigoMateria'])
            df_retorno = pd.concat([df_retorno,pd.DataFrame(row).T])
        total_escapa = total_escapa + 1
        
        if achou >= 5:
            break
        if total_escapa > 500:
            print('escapei achando',achou)
            break
    return df_retorno
    

In [ ]:
#Só precisa na primeira vez
corpus = df_detalhes_materias["combinacao"]
corpus = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower() for doc in corpus]
tokenized_corpus = [doc.split(" ") for doc in corpus]
meubm25 = BM25Okapi(tokenized_corpus)

query = "israel"
get_materias_similares_bm25(df_detalhes_materias,query,meubm25,df_detalhes_materias)

In [ ]:
bm25_scores = meubm25.get_scores('israel')
descending_indices = (-bm25_scores).argsort()
descending_indices

In [ ]:
pd.set_option('display.max_colwidth', None)
display(df_detalhes_materias[df_detalhes_materias['CodigoMateria'] == 139545]['combinacao'].to_string())

In [ ]:
bm25_scores

In [ ]:
codigo_materia = 154116
similares_pesquisadas = get_lista_similares_tramitando_base_2(cosine_sim, df_detalhes_materias, codigo_materia, df_detalhes_materias_teste)

In [ ]:
similares_pesquisadas

In [ ]:
df_detalhes_materias["combinacao"][0:4]

In [ ]:
corpus = df_detalhes_materias["combinacao"]
corpus = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower() for doc in corpus]
tokenized_corpus = [doc.split(" ") for doc in corpus]

In [ ]:
corpus[2]

In [ ]:
docs.translate(str.maketrans('', '', string.punctuation))

In [ ]:
tokenized_corpus[2]

In [ ]:
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
query = "uniforme escolar"
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
doc_scores

In [ ]:
len(doc_scores)

In [ ]:
import numpy
#sort_index = numpy.argsort(doc_scores)
#ascending_indices = doc_scores.argsort()
descending_indices = (-doc_scores).argsort()

#print(sort_index)

In [ ]:
descending_indices[0:4]

In [ ]:
df_detalhes_materias.filter(items = descending_indices[0:4], axis=0)

In [ ]:
df_detalhes_materias["combinacao"][705]

In [ ]:
#retrieve search results
query = "uniforme escolar"
tokenized_query = query.lower().split(" ")
bm25.get_top_n(tokenized_query, corpus, n=4)

In [ ]:
doc_scores = bm25.get_scores(tokenized_query)
import numpy as np
np.sort(doc_scores)[-4:][::-1]

# GERADOR DO GRÁFICO COMPARATIVO

In [ ]:
merged_1 = df_top_similares_cosseno.merge(df_top_similares_bigrama, on='CodigoMateria', how='inner')

In [ ]:
merged_2 = df_top_similares_cosseno.merge(df_top_similares_bm25, on='CodigoMateria', how='inner')

In [ ]:
merged_3 = df_top_similares_bigrama.merge(df_top_similares_bm25, on='CodigoMateria', how='inner')

In [ ]:
merged_1

In [ ]:
merged_2

In [ ]:
merged_3

In [ ]:
print("Total VERDADE",len(df_interesse_parla_verdade))
print("Total cosseno",len(df_top_similares_cosseno))
print("Total bigrama",len(df_top_similares_bigrama))
print("Total bm25",len(df_top_similares_bm25))
print("Semelhança 1:",len(merged_1))
print("Semelhança 2:",len(merged_2))
print("Semelhança 3:",len(merged_3))

In [ ]:
df_interesse_parla_verdade

In [ ]:
df_interesse_parla

In [ ]:
df_interesse_parla_treino

In [ ]:
df_interesse_parla[(df_interesse_parla['ano_materia'] >= 2023) & (df_interesse_parla['interesse'] > 0)]

In [ ]:
df_interesse_parla_verdade

In [ ]:
nlp = spacy.load('pt_core_news_lg')
from spacy.lang.pt.examples import sentences 
doc = nlp(sentences[1])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Lista de colunas para cada grupo
group1_columns = ['PercentualUnigramaNormal', 'PercentualCoberturaUnigramaLema']
group2_columns = ['PercentualCoberturaBigramaNormal', 'PercentualCoberturaBigramaLema']
group3_columns = ['PercentualCoberturaBM25Normal', 'PercentualBM25CoberturaBM25Lema']
group4_columns = ['PercentualAleatorioCobertura']

# Calcula as médias para cada grupo
group1_means = df_stat_teste[group1_columns].mean()
group2_means = df_stat_teste[group2_columns].mean()
group3_means = df_stat_teste[group3_columns].mean()
group4_means = df_stat_teste[group4_columns].mean()

# Nomes dos grupos
groups = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# Largura da barra no gráfico
bar_width = 0.2
bar_positions = np.arange(len(groups))

# Configuração do gráfico
fig, ax = plt.subplots()

bar1 = ax.bar(bar_positions - 1.5*bar_width, group1_means, bar_width, label='Group 1')
bar2 = ax.bar(bar_positions - 0.5*bar_width, group2_means, bar_width, label='Group 2')
bar3 = ax.bar(bar_positions + 0.5*bar_width, group3_means, bar_width, label='Group 3')
bar4 = ax.bar(bar_positions + 1.5*bar_width, group4_means, bar_width, label='Group 4')

# Adiciona rótulos, título e legenda
ax.set_xlabel('Groups')
ax.set_ylabel('Means')
ax.set_title('Means of Specific Columns by Group')
ax.set_xticks(bar_positions)
ax.set_xticklabels(groups)
ax.legend()

# Exibe o gráfico
plt.show()


In [ ]:

# Selecionando as colunas relevantes para cada grupo
group1 = ['PercentualCoberturaUnigramaNormal', 'PercentualCoberturaUnigramaLema']
group2 = ['PercentualCoberturaBigramaNormal', 'PercentualCoberturaBigramaLema']
group3 = ['PercentualCoberturaBM25Normal', 'PercentualBM25CoberturaBM25Lema']
group4 = ['PercentualAleatorioCobertura']

# Calculando as médias para cada grupo
means_group1 = df_stat[group1].mean()
means_group2 = df_stat[group2].mean()
means_group3 = df_stat[group3].mean()
means_group4 = df_stat[group4].mean()

# Criando o gráfico
fig, ax = plt.subplots()

bar_width = 0.15  # Reduzindo a largura das barras
group_width = 0.5  # Definindo a largura total de cada grupo

index = np.arange(len(group1) + len(group2) + len(group3) + len(group4))

bar1 = ax.bar(index[:2] - group_width/2, means_group1, bar_width, label='Cosseno - Unigrama')
bar2 = ax.bar(index[2:4] - group_width/2, means_group2, bar_width, label='Cosseno - Bigrama')
bar3 = ax.bar(index[4:6] - group_width/2, means_group3, bar_width, label='Rank BM25')
bar4 = ax.bar(index[6:] - group_width/2, means_group4, bar_width, label='Aleatório')

# Adicionando rótulos, legendas, etc.
ax.set_xlabel('Columns')
ax.set_ylabel('Mean Values')
ax.set_title('Mean Values by Column and Group')
ax.set_xticks(index - group_width/2 + bar_width/2)
#ax.set_xticklabels(group1 + group2 + group3 + group4)
#ax.set_xticklabels(group1 + group2 + group3 + group4)
plt.xticks((0, 1, 2, 3, 4, 5, 6), ('Normal', 'Lema', 'Normal', 'Lema', 'Normal', 'Lema', ''))
ax.legend()

# Exibindo o gráfico
plt.show()


In [ ]:
df_stat

In [ ]:
df_stat[df_stat['TotalBaseTreino'] < 30]

In [ ]:
len(df_stat[df_stat['TotalBaseTreino'] < 30])

In [ ]:
df_stat['PercentualAleatoriosCobertura'].mean()

In [ ]:
len(df_stat[(df_stat['TotalBaseTreino'] > 30) & (df_stat['TotalBaseVerdade'] > 10)])

In [ ]:
len(df_stat[df_stat['TotalBaseVerdade'] > 20])

In [ ]:
df_stat

In [ ]:
#df_grafico = df_stat[(df_stat['TotalBaseTreino'] > 30) & (df_stat['TotalBaseVerdade'] > 10)]
df_grafico = df_stat

# Selecionando as colunas relevantes para cada grupo
group1 = ['PercentualCoberturaUnigramaNormal','PercentualCoberturaBigramaNormal', 'PercentualCoberturaBM25Normal']          
group2 = ['PercentualCoberturaUnigramaLema','PercentualCoberturaBigramaLema','PercentualBM25CoberturaLema']
group3 = ['PercentualAleatoriosCobertura']

# Calculando as médias para cada grupo
means_group1 = df_grafico[group1].mean()
means_group2 = df_grafico[group2].mean()
means_group3 = df_grafico[group3].mean()

# Criando o gráfico
fig, ax = plt.subplots(figsize=(10, 8))

bar_width = 0.15  # Reduzindo a largura das barras
group_width = 0.2  # Definindo a largura total de cada grupo

index = np.arange(len(group1) + len(group2) + len(group3))

bar1 = ax.bar(index[:3] - group_width/2, means_group1, bar_width, label='Normal')
bar2 = ax.bar(index[3:6] - group_width/2, means_group2, bar_width, label='Lema')
bar3 = ax.bar(index[6:7] - group_width/2, means_group3, bar_width, label='Aleatório')

# Adicionando rótulos nas barras
for i, mean_value in enumerate(means_group1):
    ax.text(i - group_width/2 + bar_width/2, mean_value + 0.1, f'{mean_value:.2f}', ha='center', va='bottom')

for i, mean_value in enumerate(means_group2):
    ax.text(i + 3 - group_width/2 + bar_width/2, mean_value + 0.1, f'{mean_value:.2f}', ha='center', va='bottom')

for i, mean_value in enumerate(means_group3):
    ax.text(i + 6 - group_width/2 + bar_width/2, mean_value + 0.1, f'{mean_value:.2f}', ha='center', va='bottom')


# Adicionando rótulos, legendas, etc.
ax.set_xlabel('Algoritmos')
ax.set_ylabel('Percentual de cobertura')
ax.set_title('Percentual de cobertura dos algoritmos em relação à base de Verdade')
ax.set_xticks(index - group_width/2 + bar_width/2)
#ax.set_xticklabels(group1 + group2 + group3 + group4)
plt.xticks((0, 1, 2, 3, 4, 5, 6), ('Cosseno\n com TF-IDF', 'Cosseno\n com BoW', 'BM25', 'Cosseno\n com TF-IDF', 'Cosseno\n com BoW', 'BM25', 'Aleatório'))
ax.legend()

# Exibindo o gráfico
plt.show()


In [ ]:
index

In [ ]:
corpus_parla_normal = df_detalhes_materias["combinacao"]
corpus_parla_normal = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',' ').lower() for doc in corpus_parla_normal]
tokenized_corpus_parla_normal = [doc.split(" ") for doc in corpus_parla_normal]
bm25_parla_normal = BM25Okapi(tokenized_corpus_parla_normal)

In [ ]:
bm25_parla_normal

In [ ]:
df_sugestoes_cosseno_geral_normal

In [ ]:
df_sugestoes_bm25_geral_lema

In [ ]:
df_sugestoes_cosseno_geral_normal.merge(df_sugestoes_bm25_geral_lema, on='Identificacao', how='inner')

In [ ]:
df_sugestoes_bm25_geral_lema.to_csv('meubm25.csv')